# From Detection to Credibility: A Machine Learning Framework for Assessing News Source Reliability



In [ ]:
!pip3 install -r ../requirements.txt

In [1]:
# Import necessary libraries

# Data manipulation
import pandas as pd
import numpy as np

# Statistical functions
from scipy.stats import zscore

# For concurrency (running functions in parallel)
from concurrent.futures import ThreadPoolExecutor

# For caching (to speed up repeated function calls)
from functools import lru_cache

# For progress tracking
from tqdm import tqdm


# Text Preprocessing and NLP
import nltk
# Stopwords (common words to ignore) from NLTK
from nltk.corpus import stopwords
# Tokenizing sentences/words
from nltk.tokenize import word_tokenize
# Part-of-speech tagging
from nltk import pos_tag
# Lemmatization (converting words to their base form)
from nltk.stem import WordNetLemmatizer


# Data Preparation (Loading CSV)

Load the processed_data `csv` file into pandas DataFrames
- `processed_data.csv` is loaded into `data` DataFrame (stemming has been performed to reduce processing time.)

In [2]:
data = pd.read_csv('./processed_data.csv')

In [3]:
data['label'].value_counts()

label
0    34779
1    29081
Name: count, dtype: int64

In [4]:
data.info()
print("Dataframe Shape:", data.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63860 entries, 0 to 63859
Data columns (total 3 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   label                   63860 non-null  int64 
 1   full_content            63860 non-null  object
 2   processed_full_content  63860 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.5+ MB
Dataframe Shape: (63860, 3)


In [ ]:
# # Ensure required NLTK data is downloaded
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('wordnet')
# nltk.download('all')

### Basic Convolutional Neural Network (Tokenizer + Embedding Layer) + 5 Fold Cross-Validation + L2 Regularization

In [10]:
import tensorflow as tf
import numpy as np
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

tf.random.set_seed(42)
np.random.seed(42)
random.seed(42)

# Tokenization and Padding Parameters
max_words = 10000  # Max vocabulary size
max_sequence_length = 300  # Max length of sequences

# Tokenize and Pad Sequences
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data['processed_full_content'])
sequences = tokenizer.texts_to_sequences(data['processed_full_content'])
X = pad_sequences(sequences, maxlen=max_sequence_length)
y = data['label'].values  # Target labels

# Define the CNN Model with L2 Regularization
def create_basic_cnn():
    model = Sequential()
    model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_sequence_length))
    model.add(Conv1D(filters=128, kernel_size=5, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(Dropout(0.2))  # Add dropout for regularization
    model.add(Dense(1, activation='sigmoid', kernel_regularizer=l2(0.01)))  # Binary classification

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# 5-Fold Cross-Validation
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
fold = 1
all_fold_metrics = {'accuracy': [], 'precision': [], 'recall': [], 'f1': []}

for train_index, val_index in kf.split(X, y):
    print(f"\nTraining fold {fold}...")
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]
    
    model = create_basic_cnn()
    history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_val, y_val), verbose=1)
    
    # Predict and evaluate
    y_pred = (model.predict(X_val) > 0.5).astype(int)
    accuracy = accuracy_score(y_val, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_val, y_pred, average='binary')
    
    # Store metrics for this fold
    all_fold_metrics['accuracy'].append(accuracy)
    all_fold_metrics['precision'].append(precision)
    all_fold_metrics['recall'].append(recall)
    all_fold_metrics['f1'].append(f1)
    
    print(f"Fold {fold} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")
    fold += 1

# Calculate and print average metrics across all folds
avg_accuracy = np.mean(all_fold_metrics['accuracy'])
avg_precision = np.mean(all_fold_metrics['precision'])
avg_recall = np.mean(all_fold_metrics['recall'])
avg_f1 = np.mean(all_fold_metrics['f1'])

print("\nAverage Evaluation Metrics across 5 folds:")
print(f"Average Accuracy: {avg_accuracy:.4f}")
print(f"Average Precision: {avg_precision:.4f}")
print(f"Average Recall: {avg_recall:.4f}")
print(f"Average F1 Score: {avg_f1:.4f}")


Training fold 1...
Epoch 1/10


/home/dariusng2103/projects/dm_project/tf217/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


799/799 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8333 - loss: 0.7446 - val_accuracy: 0.9546 - val_loss: 0.2340
Epoch 2/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9613 - loss: 0.2176 - val_accuracy: 0.9641 - val_loss: 0.2012
Epoch 3/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9720 - loss: 0.1839 - val_accuracy: 0.9661 - val_loss: 0.1901
Epoch 4/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9791 - loss: 0.1630 - val_accuracy: 0.9677 - val_loss: 0.1819
Epoch 5/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9845 - loss: 0.1476 - val_accuracy: 0.9667 - val_loss: 0.1798
Epoch 6/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9879 - loss: 0.1362 - val_accuracy: 0.9660 - val_loss: 0.1760
Epoch 7/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9915 - loss: 0.1267 - val_accuracy: 0.9657 - val_loss: 0.1733
Epoch 8/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9920 - loss: 0.1199 - val_accuracy: 0.9657 - val_

/home/dariusng2103/projects/dm_project/tf217/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


799/799 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8360 - loss: 0.7398 - val_accuracy: 0.9608 - val_loss: 0.2264
Epoch 2/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9622 - loss: 0.2149 - val_accuracy: 0.9620 - val_loss: 0.2098
Epoch 3/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9732 - loss: 0.1827 - val_accuracy: 0.9663 - val_loss: 0.1945
Epoch 4/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9803 - loss: 0.1628 - val_accuracy: 0.9675 - val_loss: 0.1837
Epoch 5/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9844 - loss: 0.1485 - val_accuracy: 0.9683 - val_loss: 0.1776
Epoch 6/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9882 - loss: 0.1376 - val_accuracy: 0.9688 - val_loss: 0.1722
Epoch 7/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9908 - loss: 0.1292 - val_accuracy: 0.9690 - val_loss: 0.1681
Epoch 8/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9927 - loss: 0.1219 - val_accuracy: 0.9684 - val_

...

799/799 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8400 - loss: 0.7413 - val_accuracy: 0.9576 - val_loss: 0.2257
Epoch 2/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9597 - loss: 0.2175 - val_accuracy: 0.9645 - val_loss: 0.1986
Epoch 3/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9736 - loss: 0.1827 - val_accuracy: 0.9674 - val_loss: 0.1857
Epoch 4/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9806 - loss: 0.1625 - val_accuracy: 0.9684 - val_loss: 0.1771
Epoch 5/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9837 - loss: 0.1483 - val_accuracy: 0.9695 - val_loss: 0.1710
Epoch 6/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9873 - loss: 0.1371 - val_accuracy: 0.9700 - val_loss: 0.1665
Epoch 7/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9897 - loss: 0.1285 - val_accuracy: 0.9695 - val_loss: 0.1638
Epoch 8/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9915 - loss: 0.1212 - val_accuracy: 0.9694 - val_

/home/dariusng2103/projects/dm_project/tf217/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


799/799 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8371 - loss: 0.7514 - val_accuracy: 0.9547 - val_loss: 0.2329
Epoch 2/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9637 - loss: 0.2160 - val_accuracy: 0.9581 - val_loss: 0.2075
Epoch 3/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9738 - loss: 0.1814 - val_accuracy: 0.9605 - val_loss: 0.1974
Epoch 4/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9818 - loss: 0.1609 - val_accuracy: 0.9627 - val_loss: 0.1872
Epoch 5/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9856 - loss: 0.1457 - val_accuracy: 0.9644 - val_loss: 0.1807
Epoch 6/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9891 - loss: 0.1340 - val_accuracy: 0.9632 - val_loss: 0.1769
Epoch 7/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9911 - loss: 0.1258 - val_accuracy: 0.9635 - val_loss: 0.1743
Epoch 8/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9931 - loss: 0.1184 - val_accuracy: 0.9640 - val_

/home/dariusng2103/projects/dm_project/tf217/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


799/799 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8548 - loss: 0.7543 - val_accuracy: 0.9565 - val_loss: 0.2303
Epoch 2/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9638 - loss: 0.2168 - val_accuracy: 0.9623 - val_loss: 0.2031
Epoch 3/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9745 - loss: 0.1813 - val_accuracy: 0.9652 - val_loss: 0.1904
Epoch 4/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9819 - loss: 0.1611 - val_accuracy: 0.9657 - val_loss: 0.1842
Epoch 5/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9851 - loss: 0.1470 - val_accuracy: 0.9663 - val_loss: 0.1786
Epoch 6/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9883 - loss: 0.1364 - val_accuracy: 0.9653 - val_loss: 0.1756
Epoch 7/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9908 - loss: 0.1273 - val_accuracy: 0.9664 - val_loss: 0.1720
Epoch 8/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9917 - loss: 0.1208 - val_accuracy: 0.9663 - val_

### Convolutional Neural Network + TF-IDF Vectorizer

Using TF-IDF vectorizer along with CNN led to a drastic fall in performance. Below are some reasons why we should not use TF-IDF vectorizer along with a CNN or other neural networks.

#### Lack of Spatial Structure:

TF-IDF vectors are sparse and non-sequential representations where each position in the vector represents a word, not a spatial pattern.
CNNs are designed to detect patterns in sequential or spatially structured data (e.g., images or sentences), so they might struggle to find meaningful patterns in TF-IDF vectors.

#### High-Dimensional Sparse Data:

TF-IDF vectors, especially with a high max_features value (like 10,000), result in a high-dimensional but sparse input.
CNNs are generally not well-suited for such high-dimensional sparse data; they perform better with dense embeddings where words have contextually meaningful dimensions.

#### Mismatch Between Input Type and CNN Architecture:

CNNs are typically effective when applied to word embeddings (like GloVe or Word2Vec) because embeddings maintain semantic relationships and neighborhood structures.
TF-IDF, however, does not capture word order or semantic relationships, which means the convolution operation might not yield meaningful feature maps.


In [ ]:
import tensorflow as tf
import numpy as np
import random

tf.random.set_seed(42)
np.random.seed(42)
random.seed(42)

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, Dense, Dropout, Reshape, Input
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Step 1: Apply TF-IDF Vectorization
max_features = 10000  # Limit TF-IDF to top 10,000 features
tfidf_vectorizer = TfidfVectorizer(max_features=max_features)
X_tfidf = tfidf_vectorizer.fit_transform(data['processed_full_content']).toarray()

# Convert the labels
y = data['label'].values  # Target labels

# Step 2: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Step 3: Define the CNN Model for TF-IDF Input
def create_cnn_with_tfidf():
    inputs = Input(shape=(max_features,))
    x = Reshape((max_features, 1))(inputs)  # Reshape TF-IDF output to be compatible with Conv1D

    # Convolutional layer
    x = Conv1D(filters=128, kernel_size=5, activation='relu')(x)
    x = GlobalMaxPooling1D()(x)
    
    # Fully connected layer
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.5)(x)  # Dropout for regularization
    outputs = Dense(1, activation='sigmoid')(x)  # Output layer for binary classification

    # Create model
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Step 4: Train the Model
model = create_cnn_with_tfidf()
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test), verbose=1)

# Step 5: Evaluate the Model
y_pred = (model.predict(X_test) > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='binary')

print("\nEvaluation Metrics:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Epoch 1/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 24s 24ms/step - accuracy: 0.5353 - loss: 0.6893 - val_accuracy: 0.5483 - val_loss: 0.6825
Epoch 2/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - accuracy: 0.5550 - loss: 0.6836 - val_accuracy: 0.5729 - val_loss: 0.6786
Epoch 3/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 10s 13ms/step - accuracy: 0.5565 - loss: 0.6815 - val_accuracy: 0.5727 - val_loss: 0.6772
Epoch 4/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 11s 14ms/step - accuracy: 0.5610 - loss: 0.6801 - val_accuracy: 0.5727 - val_loss: 0.6752
Epoch 5/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - accuracy: 0.5593 - loss: 0.6793 - val_accuracy: 0.5710 - val_loss: 0.6766
Epoch 6/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 10s 13ms/step - accuracy: 0.5620 - loss: 0.6795 - val_accuracy: 0.5711 - val_loss: 0.6749
Epoch 7/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 11s 14ms/step - accuracy: 0.5590 - loss: 0.6791 - val_accuracy: 0.5730 - val_loss: 0.6756
Epoch 8/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - accuracy: 0.5605 - loss: 0.6782 - val

### Convolutional Neural Networks + Count Vectorization (conversion to sequences) + Stratified 5-Fold CV + L2 Regularization

In [9]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tensorflow.keras.regularizers import l2

seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)
random.seed(seed)

# Step 1: Text Vectorization using CountVectorizer
max_features = 10000  # Max vocabulary size for CountVectorizer
vectorizer = CountVectorizer(max_features=max_features)
X_counts = vectorizer.fit_transform(data['processed_full_content'])
word_index = vectorizer.vocabulary_

# Convert CountVectorizer output to sequences
index_to_word = {i: word for word, i in word_index.items()}

def counts_to_sequences(X_counts):
    sequences = []
    for i in range(X_counts.shape[0]):
        indices = X_counts[i].nonzero()[1]
        words = [index_to_word[idx] for idx in indices]
        seq = [word_index[word] + 1 for word in words]  # +1 because 0 is reserved for padding
        sequences.append(seq)
    return sequences

sequences = counts_to_sequences(X_counts)
max_sequence_length = 300  # Adjust to your needs
X = pad_sequences(sequences, maxlen=max_sequence_length)
y = data['label'].values  # Target labels

# Define the Basic CNN Model with L2 Regularization
def create_basic_cnn_with_l2():
    model = Sequential()
    
    # Embedding layer with random initialization
    model.add(Embedding(input_dim=max_features + 1, output_dim=128))
    
    # Convolutional layer with L2 regularization
    model.add(Conv1D(filters=128, kernel_size=5, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(GlobalMaxPooling1D())
    
    # Fully connected layer with L2 regularization
    model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(Dropout(0.5))  # Add dropout for regularization
    
    # Output layer with L2 regularization
    model.add(Dense(1, activation='sigmoid', kernel_regularizer=l2(0.01)))  # Binary classification
    
    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Step 4: Stratified 5-Fold Cross-Validation
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []

for train_index, test_index in kf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    model = create_basic_cnn_with_l2()
    model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2, verbose=1)
    
    # Evaluate the model
    y_pred = (model.predict(X_test) > 0.5).astype(int)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='binary')
    
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)

# Step 5: Print Cross-Validation Results
print("\nCross-Validation Metrics:")
print(f"Average Accuracy: {np.mean(accuracy_scores):.4f}")
print(f"Average Precision: {np.mean(precision_scores):.4f}")
print(f"Average Recall: {np.mean(recall_scores):.4f}")
print(f"Average F1 Score: {np.mean(f1_scores):.4f}")


Epoch 1/10
639/639 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7685 - loss: 0.8773 - val_accuracy: 0.9291 - val_loss: 0.2726
Epoch 2/10
639/639 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9301 - loss: 0.2740 - val_accuracy: 0.9330 - val_loss: 0.2610
Epoch 3/10
639/639 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9425 - loss: 0.2451 - val_accuracy: 0.9389 - val_loss: 0.2481
Epoch 4/10
639/639 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9531 - loss: 0.2220 - val_accuracy: 0.9424 - val_loss: 0.2419
Epoch 5/10
639/639 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9605 - loss: 0.2073 - val_accuracy: 0.9360 - val_loss: 0.2567
Epoch 6/10
639/639 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9662 - loss: 0.1931 - val_accuracy: 0.9419 - val_loss: 0.2467
Epoch 7/10
639/639 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9734 - loss: 0.1802 - val_accuracy: 0.9399 - val_loss: 0.2534
Epoch 8/10
639/639 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9773 - loss: 0.1705 - val_accuracy: 0.

### Convolutional Neural Network + Custom-trained Word2Vec Embeddings + 5-Fold Cross Validation + L2 Regularization

#### Why do we use word embedding over other preprocessing techniques (eg. tf-idf, count vectorizer), for our task of fake news classification?


##### 1. Word embeddings capture the semantic relationships between words in a dense, low-dimensional space.
Fake news often uses subtle language, and word embeddings like GloVe can capture the semantic context of words, allowing the model to understand relationships between words that simple vectorizers would miss. This helps in detecting nuanced differences in language use between real and fake news.

##### 2. Word embeddings produce dense, low-dimensional vectors (e.g., 100-300 dimensions) that capture rich word information.
Pre-trained embeddings are built on large corpora like Wikipedia and news articles, giving our model external knowledge that’s useful for distinguishing between real news and fake news. This boosts the model's ability to generalize on unseen test data from our web scraping.

##### 3. Efficient Representation of Semantics
Words in fake news can appear in different contexts, but with similar underlying meanings (e.g., "hoax" and "lie"). GloVe embeddings represent these similar words in close proximity in the vector space, helping the model recognize fake news patterns more effectively than TF-IDF or Count Vectorizer.

##### 4. Handling Synonyms and Rare Words:
Fake news often uses alternative phrases or rare terminology. Pre-trained embeddings like GloVe can handle these rare words because they’ve seen a broad variety of language during training, making our model more robust against unusual vocabulary choices in fake news.

## Cross Validation 
We use Stratified K-Fold Cross-Validation with n_splits=5 to evaluate the model on different splits of the data. 
For each fold, we store the metrics (accuracy, precision, recall, and F1 score) and then calculate the average metrics across all folds for a robust evaluation.

Cross-validation helps us understand the model’s performance more robustly by testing it on multiple splits of the data. This approach gives a more reliable estimate of model performance and helps reduce the risk of overfitting to any single train-test split.

In [5]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout, Input
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from gensim.models import Word2Vec
from tensorflow.keras.regularizers import l2
import random

# Set seeds
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)
random.seed(seed)

# Parameters
max_words = 5000
max_sequence_length = 300
embedding_dim = 100

def create_embedding_matrix(word2vec_model, tokenizer, vocab_size, embedding_dim):
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    for word, i in tokenizer.word_index.items():
        if i < vocab_size:
            if word in word2vec_model.wv:
                embedding_matrix[i] = word2vec_model.wv[word]
            else:
                embedding_matrix[i] = np.random.normal(size=(embedding_dim,))
    return embedding_matrix

def create_cnn_with_l2(vocab_size, embedding_dim, embedding_matrix):
    input_layer = Input(shape=(max_sequence_length,))
    embedding_layer = Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dim,
        weights=[embedding_matrix],
        trainable=True
    )(input_layer)
    
    x = Conv1D(filters=128, kernel_size=5, activation='relu', kernel_regularizer=l2(0.01))(embedding_layer)
    x = GlobalMaxPooling1D()(x)
    x = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(x)
    x = Dropout(0.5)(x)
    output_layer = Dense(1, activation='sigmoid', kernel_regularizer=l2(0.01))(x)
    
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Implement cross-validation
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
fold_metrics = []

for fold, (train_idx, val_idx) in enumerate(kfold.split(data['processed_full_content'], data['label']), 1):
    print(f"\nFold {fold}")
    
    # Split data
    train_texts = data['processed_full_content'].iloc[train_idx]
    val_texts = data['processed_full_content'].iloc[val_idx]
    train_labels = data['label'].iloc[train_idx]
    val_labels = data['label'].iloc[val_idx]
    
    # Tokenization
    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(train_texts)
    
    # Create sequences
    X_train = pad_sequences(tokenizer.texts_to_sequences(train_texts), maxlen=max_sequence_length)
    X_val = pad_sequences(tokenizer.texts_to_sequences(val_texts), maxlen=max_sequence_length)
    
    # Train Word2Vec on training data only
    train_sentences = [text.split() for text in train_texts]
    word2vec_model = Word2Vec(train_sentences, vector_size=embedding_dim, window=5, min_count=2, workers=4)
    
    # Create embedding matrix
    vocab_size = min(max_words, len(tokenizer.word_index) + 1)
    embedding_matrix = create_embedding_matrix(word2vec_model, tokenizer, vocab_size, embedding_dim)
    
    # Create and train model
    model = create_cnn_with_l2(vocab_size, embedding_dim, embedding_matrix)
    
    history = model.fit(
        X_train, train_labels,
        epochs=10,
        batch_size=256,
        validation_data=(X_val, val_labels),
        verbose=1
    )
    
    # Evaluate
    y_pred = (model.predict(X_val) > 0.5).astype(int)
    
    # Calculate metrics
    accuracy = accuracy_score(val_labels, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(val_labels, y_pred, average='binary')
    
    fold_metrics.append({
        'fold': fold,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    })
    
    print(f"\nFold {fold} Results:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

# Calculate and display average metrics
avg_metrics = {
    'accuracy': np.mean([m['accuracy'] for m in fold_metrics]),
    'precision': np.mean([m['precision'] for m in fold_metrics]),
    'recall': np.mean([m['recall'] for m in fold_metrics]),
    'f1': np.mean([m['f1'] for m in fold_metrics])
}

print("\nAverage Metrics Across All Folds:")
print(f"Average Accuracy: {avg_metrics['accuracy']:.4f}")
print(f"Average Precision: {avg_metrics['precision']:.4f}")
print(f"Average Recall: {avg_metrics['recall']:.4f}")
print(f"Average F1 Score: {avg_metrics['f1']:.4f}")

2024-11-09 01:12:42.731442: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-09 01:12:42.765781: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731085962.782279  239807 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731085962.785495  239807 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-09 01:12:42.807291: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr


Fold 1


I0000 00:00:1731086003.944731  239807 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3733 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070, pci bus id: 0000:01:00.0, compute capability: 8.9


Epoch 1/10


I0000 00:00:1731086005.956369  240355 service.cc:148] XLA service 0x7f9c4c016260 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1731086005.956407  240355 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4070, Compute Capability 8.9
2024-11-09 01:13:25.974511: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1731086006.051255  240355 cuda_dnn.cc:529] Loaded cuDNN version 90300


 18/200 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5649 - loss: 3.4362

I0000 00:00:1731086006.031127  240355 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.8072 - loss: 2.0391 - val_accuracy: 0.9526 - val_loss: 0.7056
Epoch 2/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9505 - loss: 0.6093 - val_accuracy: 0.9627 - val_loss: 0.3841
Epoch 3/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9626 - loss: 0.3599 - val_accuracy: 0.9620 - val_loss: 0.2805
Epoch 4/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9670 - loss: 0.2621 - val_accuracy: 0.9682 - val_loss: 0.2207
Epoch 5/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9709 - loss: 0.2118 - val_accuracy: 0.9694 - val_loss: 0.1917
Epoch 6/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9739 - loss: 0.1836 - val_accuracy: 0.9711 - val_loss: 0.1752
Epoch 7/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9744 - loss: 0.1696 - val_accuracy: 0.9726 - val_loss: 0.1662
Epoch 8/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9779 - loss: 0.1575 - val_accuracy: 0.9717 - val

### Convolutional Neural network + GloVe word embeddings (100D) + 5-Fold Cross Validation + L2 regularization

In [13]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout, Concatenate, Input
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from sklearn.utils.class_weight import compute_class_weight

# Set seeds
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)
random.seed(seed)

# Load GloVe embeddings once (this doesn't cause data leakage)
def load_glove_embeddings(path, embedding_dim=100):
    print("Loading GloVe embeddings...")
    embeddings_index = {}
    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    print(f"Loaded {len(embeddings_index)} word vectors.")
    return embeddings_index

def create_embedding_matrix(word_index, embeddings_index, vocab_size, embedding_dim):
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    for word, i in word_index.items():
        if i >= vocab_size:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

def create_model(vocab_size, embedding_matrix, max_sequence_length):
    input_layer = Input(shape=(max_sequence_length,))
    embedding_layer = Embedding(
        input_dim=vocab_size,
        output_dim=embedding_matrix.shape[1],
        weights=[embedding_matrix],
        trainable=False,  # Set to False for pre-trained embeddings
        input_length=max_sequence_length
    )(input_layer)

    convs = []
    for kernel_size in [3, 4, 5]:
        conv = Conv1D(
            filters=64,
            kernel_size=kernel_size,
            activation='relu',
            kernel_regularizer=l2(0.01)
        )(embedding_layer)
        pool = GlobalMaxPooling1D()(conv)
        convs.append(pool)

    merged = Concatenate()(convs)
    dense = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(merged)
    drop = Dropout(0.2)(dense)
    output = Dense(1, activation='sigmoid', kernel_regularizer=l2(0.01))(drop)

    model = Model(inputs=input_layer, outputs=output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def process_fold_data(train_texts, val_texts, tokenizer, max_sequence_length):
    """Process text data for a single fold"""
    # Fit tokenizer on training data only
    tokenizer.fit_on_texts(train_texts)
    
    # Convert texts to sequences
    X_train = tokenizer.texts_to_sequences(train_texts)
    X_val = tokenizer.texts_to_sequences(val_texts)
    
    # Pad sequences
    X_train = pad_sequences(X_train, maxlen=max_sequence_length)
    X_val = pad_sequences(X_val, maxlen=max_sequence_length)
    
    return X_train, X_val, tokenizer

def main():
    # Parameters
    max_sequence_length = 300
    vocab_size = 5000
    embedding_dim = 100
    
    # Load GloVe embeddings
    glove_path = '../glove.6B.100d.txt'
    embeddings_index = load_glove_embeddings(glove_path, embedding_dim)
    
    # Prepare for cross-validation
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    fold_metrics = []
    
    for fold, (train_idx, val_idx) in enumerate(kfold.split(data['processed_full_content'], data['label']), 1):
        print(f"\nFold {fold}")
        
        # Split data
        train_texts = data['processed_full_content'].iloc[train_idx]
        val_texts = data['processed_full_content'].iloc[val_idx]
        y_train = data['label'].iloc[train_idx].values
        y_val = data['label'].iloc[val_idx].values
        
        # Initialize new tokenizer for each fold
        tokenizer = Tokenizer(num_words=vocab_size)
        
        # Process data for this fold
        X_train, X_val, tokenizer = process_fold_data(
            train_texts, val_texts, tokenizer, max_sequence_length
        )
        
        # Create embedding matrix for this fold's vocabulary
        embedding_matrix = create_embedding_matrix(
            tokenizer.word_index, embeddings_index, vocab_size, embedding_dim
        )
        
        # Compute class weights
        class_weights = compute_class_weight('balanced', 
                                          classes=np.unique(y_train), 
                                          y=y_train)
        class_weights_dict = dict(enumerate(class_weights))
        
        # Create and train model
        model = create_model(vocab_size, embedding_matrix, max_sequence_length)
        
        # Train the model
        history = model.fit(
            X_train, y_train,
            epochs=10,
            batch_size=256,
            validation_data=(X_val, y_val),
            class_weight=class_weights_dict,
            verbose=1
        )
        
        # Evaluate
        y_pred = (model.predict(X_val) > 0.5).astype(int)
        
        # Calculate metrics
        accuracy = accuracy_score(y_val, y_pred)
        precision, recall, f1, _ = precision_recall_fscore_support(y_val, y_pred, average='binary')
        
        fold_metrics.append({
            'fold': fold,
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1
        })
        
        print(f"\nFold {fold} Results:")
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1-score: {f1:.4f}")
    
    # Calculate and print average metrics
    avg_metrics = {
        'accuracy': np.mean([m['accuracy'] for m in fold_metrics]),
        'precision': np.mean([m['precision'] for m in fold_metrics]),
        'recall': np.mean([m['recall'] for m in fold_metrics]),
        'f1': np.mean([m['f1'] for m in fold_metrics])
    }
    
    print("\nAverage Metrics Across All Folds:")
    for metric, value in avg_metrics.items():
        print(f"{metric.capitalize()}: {value:.4f}")

if __name__ == "__main__":
    main()

Loading GloVe embeddings...
Loaded 400000 word vectors.

Fold 1
Epoch 1/10


/home/dariusng2103/projects/dm_project/tf217/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


197/200 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7661 - loss: 2.4662

2024-11-08 19:43:17.599910: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1335', 64 bytes spill stores, 64 bytes spill loads



200/200 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - accuracy: 0.7680 - loss: 2.4525 - val_accuracy: 0.9388 - val_loss: 0.6112
Epoch 2/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9400 - loss: 0.5312 - val_accuracy: 0.9515 - val_loss: 0.3739
Epoch 3/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9515 - loss: 0.3519 - val_accuracy: 0.9534 - val_loss: 0.3055
Epoch 4/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9526 - loss: 0.2967 - val_accuracy: 0.9557 - val_loss: 0.2781
Epoch 5/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9565 - loss: 0.2745 - val_accuracy: 0.9576 - val_loss: 0.2680
Epoch 6/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9570 - loss: 0.2671 - val_accuracy: 0.9568 - val_loss: 0.2663
Epoch 7/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9565 - loss: 0.2640 - val_accuracy: 0.9573 - val_loss: 0.2632
Epoch 8/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9587 - loss: 0.2603 - val_accuracy: 0.9576 - val

...

200/200 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.7472 - loss: 2.4114 - val_accuracy: 0.9346 - val_loss: 0.5573
Epoch 2/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9382 - loss: 0.4881 - val_accuracy: 0.9515 - val_loss: 0.3466
Epoch 3/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9514 - loss: 0.3351 - val_accuracy: 0.9556 - val_loss: 0.2885
Epoch 4/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9547 - loss: 0.2896 - val_accuracy: 0.9573 - val_loss: 0.2682
Epoch 5/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9544 - loss: 0.2744 - val_accuracy: 0.9566 - val_loss: 0.2627
Epoch 6/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9566 - loss: 0.2677 - val_accuracy: 0.9579 - val_loss: 0.2581
Epoch 7/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9572 - loss: 0.2636 - val_accuracy: 0.9588 - val_loss: 0.2552
Epoch 8/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9580 - loss: 0.2614 - val_accuracy: 0.9584 - val

/home/dariusng2103/projects/dm_project/tf217/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


200/200 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.7541 - loss: 2.3498 - val_accuracy: 0.9434 - val_loss: 0.5372
Epoch 2/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9415 - loss: 0.4749 - val_accuracy: 0.9509 - val_loss: 0.3436
Epoch 3/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9514 - loss: 0.3298 - val_accuracy: 0.9553 - val_loss: 0.2884
Epoch 4/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9555 - loss: 0.2855 - val_accuracy: 0.9558 - val_loss: 0.2737
Epoch 5/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9556 - loss: 0.2717 - val_accuracy: 0.9541 - val_loss: 0.2698
Epoch 6/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9554 - loss: 0.2654 - val_accuracy: 0.9537 - val_loss: 0.2687
Epoch 7/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9571 - loss: 0.2632 - val_accuracy: 0.9549 - val_loss: 0.2659
Epoch 8/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9579 - loss: 0.2600 - val_accuracy: 0.9559 - val

/home/dariusng2103/projects/dm_project/tf217/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


200/200 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.7615 - loss: 2.3720 - val_accuracy: 0.9369 - val_loss: 0.5430
Epoch 2/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9396 - loss: 0.4779 - val_accuracy: 0.9518 - val_loss: 0.3464
Epoch 3/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9498 - loss: 0.3382 - val_accuracy: 0.9534 - val_loss: 0.2930
Epoch 4/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9554 - loss: 0.2891 - val_accuracy: 0.9548 - val_loss: 0.2718
Epoch 5/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9566 - loss: 0.2731 - val_accuracy: 0.9563 - val_loss: 0.2626
Epoch 6/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9600 - loss: 0.2632 - val_accuracy: 0.9573 - val_loss: 0.2592
Epoch 7/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9593 - loss: 0.2610 - val_accuracy: 0.9566 - val_loss: 0.2590
Epoch 8/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9592 - loss: 0.2591 - val_accuracy: 0.9576 - val

### Convolutional Neural network + Custom-trained word2vec word embeddings + 5-Fold Cross Validation + L2 Regularization + GridSearchCV


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout, Input
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from gensim.models import Word2Vec
from tensorflow.keras.regularizers import l2

# Set seeds for reproducibility
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

def train_word2vec_and_create_embeddings(train_texts, word_index, max_words, embedding_dim=100):
    """Train Word2Vec on training data only and create embedding matrix"""
    # Train Word2Vec on training data only
    train_sentences = [text.split() for text in train_texts]
    word2vec_model = Word2Vec(sentences=train_sentences, 
                            vector_size=embedding_dim, 
                            window=5, 
                            min_count=2, 
                            workers=4)
    
    # Create embedding matrix with correct dimensions
    vocab_size = min(max_words, len(word_index) + 1)
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    
    for word, i in word_index.items():
        if i < vocab_size:  # Only include words within max_words limit
            if word in word2vec_model.wv:
                embedding_matrix[i] = word2vec_model.wv[word]
            else:
                embedding_matrix[i] = np.random.normal(size=(embedding_dim,))
            
    return embedding_matrix

def create_model(max_sequence_length, vocab_size, embedding_dim, embedding_matrix, 
                filters, dropout_rate):
    input_layer = Input(shape=(max_sequence_length,))
    embedding_layer = Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dim,
        weights=[embedding_matrix],
        trainable=True
    )(input_layer)

    x = Conv1D(
        filters=filters,
        kernel_size=5,
        activation='relu',
        kernel_regularizer=l2(0.01)
    )(embedding_layer)
    
    x = GlobalMaxPooling1D()(x)
    x = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(x)
    x = Dropout(dropout_rate)(x)
    output_layer = Dense(1, activation='sigmoid', kernel_regularizer=l2(0.01))(x)

    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def main():
    # Define parameter grid
    param_grid = {
        'filters': [64, 128],
        'dropout_rate': [0.2, 0.3, 0.4, 0.5]
    }

    # Initialize variables to track results
    results = []
    best_score = 0
    best_params = None

    # Constants
    max_words = 10000
    max_sequence_length = 300
    embedding_dim = 100

    # Perform grid search with cross-validation
    for filters in param_grid['filters']:
        for dropout_rate in param_grid['dropout_rate']:
            print(f"\nTesting filters={filters}, dropout_rate={dropout_rate}")
            
            # Initialize cross-validation
            kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
            fold_scores = []
            
            # Perform k-fold cross-validation
            for fold, (train_idx, val_idx) in enumerate(kfold.split(data['processed_full_content'], data['label']), 1):
                print(f"\nFold {fold}")
                
                # Split data
                train_texts = data['processed_full_content'].iloc[train_idx]
                val_texts = data['processed_full_content'].iloc[val_idx]
                y_train = data['label'].iloc[train_idx]
                y_val = data['label'].iloc[val_idx]
                
                # Fit tokenizer on training data only
                tokenizer = Tokenizer(num_words=max_words)
                tokenizer.fit_on_texts(train_texts)
                
                # Convert texts to sequences
                X_train = pad_sequences(tokenizer.texts_to_sequences(train_texts), 
                                      maxlen=max_sequence_length)
                X_val = pad_sequences(tokenizer.texts_to_sequences(val_texts), 
                                    maxlen=max_sequence_length)
                
                # Get vocab size for this fold
                vocab_size = min(max_words, len(tokenizer.word_index) + 1)
                
                # Create embedding matrix using training data only
                embedding_matrix = train_word2vec_and_create_embeddings(
                    train_texts, 
                    tokenizer.word_index,
                    max_words,
                    embedding_dim
                )
                
                # Create and train model
                model = create_model(
                    max_sequence_length=max_sequence_length,
                    vocab_size=vocab_size,
                    embedding_dim=embedding_dim,
                    embedding_matrix=embedding_matrix,
                    filters=filters,
                    dropout_rate=dropout_rate
                )
                
                # Train model
                history = model.fit(
                    X_train, y_train,
                    epochs=10,
                    batch_size=64,
                    validation_data=(X_val, y_val),
                    verbose=1
                )
                
                # Evaluate using F1-score
                y_pred = (model.predict(X_val) > 0.5).astype(int)
                fold_score = f1_score(y_val, y_pred)
                fold_scores.append(fold_score)
                
                print(f"Fold {fold} F1-score: {fold_score:.4f}")
            
            # Calculate average score for this parameter combination
            avg_score = np.mean(fold_scores)
            print(f"Average F1-score: {avg_score:.4f}")
            
            # Store results
            results.append({
                'filters': filters,
                'dropout_rate': dropout_rate,
                'avg_f1_score': avg_score,
                'fold_scores': fold_scores
            })
            
            # Update best parameters if necessary
            if avg_score > best_score:
                best_score = avg_score
                best_params = {'filters': filters, 'dropout_rate': dropout_rate}

    # Print final results
    print("\nGrid Search Results:")
    for result in results:
        print(f"Filters: {result['filters']}, Dropout: {result['dropout_rate']}, "
              f"F1-score: {result['avg_f1_score']:.4f}")

    print("\nBest Parameters:")
    print(f"Filters: {best_params['filters']}")
    print(f"Dropout Rate: {best_params['dropout_rate']}")
    print(f"Best F1-Score: {best_score:.4f}")

    # Save results to DataFrame for easy analysis
    import pandas as pd
    results_df = pd.DataFrame(results)
    print("\nResults Summary:")
    print(results_df.sort_values('avg_f1_score', ascending=False))

if __name__ == "__main__":
    main()


Testing filters=64, dropout_rate=0.2

Fold 1
Epoch 1/10


/home/dariusng2103/projects/dm_project/tf217/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


799/799 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8696 - loss: 1.1564 - val_accuracy: 0.9597 - val_loss: 0.3197
Epoch 2/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9587 - loss: 0.2911 - val_accuracy: 0.9637 - val_loss: 0.2099
Epoch 3/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9653 - loss: 0.2014 - val_accuracy: 0.9672 - val_loss: 0.1779
Epoch 4/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9708 - loss: 0.1702 - val_accuracy: 0.9681 - val_loss: 0.1686
Epoch 5/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9732 - loss: 0.1569 - val_accuracy: 0.9721 - val_loss: 0.1545
Epoch 6/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9772 - loss: 0.1432 - val_accuracy: 0.9727 - val_loss: 0.1525
Epoch 7/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9811 - loss: 0.1357 - val_accuracy: 0.9709 - val_loss: 0.1501
Epoch 8/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9833 - loss: 0.1280 - val_accuracy: 0.9721 - val_

/home/dariusng2103/projects/dm_project/tf217/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


799/799 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8592 - loss: 1.1943 - val_accuracy: 0.9619 - val_loss: 0.3178
Epoch 2/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9594 - loss: 0.2835 - val_accuracy: 0.9663 - val_loss: 0.2057
Epoch 3/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9666 - loss: 0.1958 - val_accuracy: 0.9652 - val_loss: 0.1847
Epoch 4/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9702 - loss: 0.1692 - val_accuracy: 0.9644 - val_loss: 0.1782
Epoch 5/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9728 - loss: 0.1565 - val_accuracy: 0.9649 - val_loss: 0.1728
Epoch 6/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9755 - loss: 0.1456 - val_accuracy: 0.9655 - val_loss: 0.1695
Epoch 7/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9792 - loss: 0.1361 - val_accuracy: 0.9683 - val_loss: 0.1626
Epoch 8/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9819 - loss: 0.1306 - val_accuracy: 0.9713 - val_

...

/home/dariusng2103/projects/dm_project/tf217/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


799/799 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8626 - loss: 1.4213 - val_accuracy: 0.9543 - val_loss: 0.3309
Epoch 2/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9534 - loss: 0.3031 - val_accuracy: 0.9619 - val_loss: 0.2137
Epoch 3/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9632 - loss: 0.2110 - val_accuracy: 0.9656 - val_loss: 0.1868
Epoch 4/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9686 - loss: 0.1824 - val_accuracy: 0.9695 - val_loss: 0.1708
Epoch 5/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9726 - loss: 0.1657 - val_accuracy: 0.9706 - val_loss: 0.1612
Epoch 6/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9762 - loss: 0.1515 - val_accuracy: 0.9713 - val_loss: 0.1551
Epoch 7/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9774 - loss: 0.1419 - val_accuracy: 0.9717 - val_loss: 0.1519
Epoch 8/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9802 - loss: 0.1361 - val_accuracy: 0.9728 - val_

/home/dariusng2103/projects/dm_project/tf217/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


799/799 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8681 - loss: 1.4816 - val_accuracy: 0.9569 - val_loss: 0.3455
Epoch 2/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9578 - loss: 0.3058 - val_accuracy: 0.9666 - val_loss: 0.2015
Epoch 3/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9648 - loss: 0.2027 - val_accuracy: 0.9676 - val_loss: 0.1790
Epoch 4/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9692 - loss: 0.1761 - val_accuracy: 0.9672 - val_loss: 0.1685
Epoch 5/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 636us/step - accuracy: 0.9728 - loss: 0.1626 - val_accuracy: 0.9711 - val_loss: 0.1569
Epoch 6/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9763 - loss: 0.1491 - val_accuracy: 0.9719 - val_loss: 0.1520
Epoch 7/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9801 - loss: 0.1407 - val_accuracy: 0.9714 - val_loss: 0.1507
Epoch 8/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9811 - loss: 0.1351 - val_accuracy: 0.9730 - va

: 